In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [8]:
dataset_url = "http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip"
csv_file = "datasets/petfinder-mini/petfinder-mini.csv"

tf.keras.utils.get_file('petfinder-mini.csv', dataset_url, extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [9]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [10]:
dataframe.dtypes

Type             object
Age               int64
Breed1           object
Gender           object
Color1           object
Color2           object
MaturitySize     object
FurLength        object
Vaccinated       object
Sterilized       object
Health           object
Fee               int64
Description      object
PhotoAmt          int64
AdoptionSpeed     int64
dtype: object

In [13]:
dataframe['target'] = np.where(dataframe['AdoptionSpeed'] == 4, 0, 1)
dataframe = dataframe.drop(columns=["AdoptionSpeed", "Description"])

In [14]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

7383 train examples
1846 validation examples
2308 test examples


In [15]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [16]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

2022-09-30 14:48:15.959407: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-09-30 14:48:15.983279: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb6b757a7b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-09-30 14:48:15.983310: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [18]:
for feature_batch, label_batch in train_ds.take(1):
    print("Every feature:", list(feature_batch.keys()))
    print("A batch of ages:", feature_batch['Age'])
    print("A batch of targets:", label_batch)

Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([ 1 48  4  3  2], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 0 0 0 1], shape=(5,), dtype=int64)


In [19]:
example_batch = next(iter(train_ds))[0]

In [20]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [23]:
age = feature_column.numeric_column("PhotoAmt")
demo(age)

[[2.]
 [5.]
 [6.]
 [3.]
 [1.]]


In [24]:
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 3, 5])
demo(age_buckets)

[[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]


In [25]:
animal_type = feature_column.categorical_column_with_vocabulary_list('Type', ['Cat', 'Dog'])
animal_type_one_hot = feature_column.indicator_column(animal_type)
demo(animal_type_one_hot)

[[0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [30]:
breed1 = feature_column.categorical_column_with_vocabulary_list(
      'Breed1', dataframe.Breed1.unique())
breed1_embedding = feature_column.embedding_column(breed1, dimension=8)
demo(breed1_embedding)

[[-0.6955944   0.41754076 -0.29764265  0.11198223  0.24526243  0.03003561
  -0.6131082  -0.28161636]
 [ 0.1532884   0.23392639  0.3941193   0.2815165   0.1058752   0.30130625
  -0.26738662 -0.12063981]
 [ 0.6458973  -0.49344856 -0.15890576 -0.06708071 -0.08721107  0.58078456
   0.03834648 -0.6816039 ]
 [-0.29299992 -0.14519414  0.06181984  0.22504573 -0.19697309  0.27053517
  -0.33197734  0.38882944]
 [ 0.07877477  0.30034196  0.30848712  0.22877495 -0.69072634  0.63963836
   0.38317323  0.08114266]]


In [32]:
breed1_hashed = feature_column.categorical_column_with_hash_bucket(
      'Breed1', hash_bucket_size=10)
demo(feature_column.indicator_column(breed1_hashed))

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [34]:
cross_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=10)
demo(feature_column.indicator_column(cross_feature))

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]


In [65]:
feature_columns = []

for header in ['PhotoAmt', 'Fee', 'Age']:
    feature_columns.append(feature_column.numeric_column(header))

In [66]:

age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1,2,3,4,5])
feature_columns.append(age_buckets)

In [67]:
indicator_column_names = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health']
for column_name in indicator_column_names:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(column_name, dataframe[column_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)

In [68]:
bread1 = feature_column.categorical_column_with_vocabulary_list('Breed1', dataframe.Breed1.unique())
bread1_embedding = feature_column.embedding_column(bread1, dimension=8)
feature_columns.append(bread1_embedding)

In [69]:
animal_type = feature_column.categorical_column_with_vocabulary_list(
      'Type', ['Cat', 'Dog'])
age_type_feature = feature_column.crossed_column([age_buckets, animal_type], hash_bucket_size=100)
feature_columns.append(feature_column.indicator_column(age_type_feature))

In [70]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [71]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [74]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
231/231 [==============================] - 1s 3ms/step - loss: 0.6247 - accuracy: 0.6980 - val_loss: 0.5456 - val_accuracy: 0.7454
Epoch 2/10
231/231 [==============================] - 0s 2ms/step - loss: 0.5534 - accuracy: 0.7295 - val_loss: 0.5669 - val_accuracy: 0.6950
Epoch 3/10
231/231 [==============================] - 0s 2ms/step - loss: 0.5055 - accuracy: 0.7375 - val_loss: 0.5083 - val_accuracy: 0.7226
Epoch 4/10
231/231 [==============================] - 0s 2ms/step - loss: 0.4911 - accuracy: 0.7448 - val_loss: 0.5183 - val_accuracy: 0.7557
Epoch 5/10
231/231 [==============================] - 0s 2ms/step - loss: 0.4844 - accuracy: 0.7486 - val_loss: 0.5053 - val_accuracy: 0.7373
Epoch 6/10
231/231 [==============================] - 0s 2ms/step - loss: 0.4726 - accuracy: 0.7581 - val_loss: 0.5161 - val_accuracy: 0.7313
Epoch 7/10
231/231 [==============================] - 0s 2ms/step - loss: 0.4735 - accuracy: 0.7548 - val_loss: 0.5147 - val_accuracy: 0.7584
Epoch 

In [75]:
loss, accuracy = model.evaluate(test_ds)
print('Accuracy', accuracy)

73/73 [==============================] - 0s 2ms/step - loss: 0.5066 - accuracy: 0.7175
Accuracy 0.7175043225288391
